# Distribución Metros Cuadrados Construidos


In [7]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    if nombre.startswith("df_") and "alfa" in nombre and "queretaro" in nombre or 'df_junio_2023_queretaro' in nombre
]
# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_abril_2024_queretaro',
 'df_alfa_agosto_2024_queretaro',
 'df_alfa_marzo_2024_queretaro',
 'df_alfa_mayo_2024_queretaro',
 'df_junio_2023_queretaro']

In [8]:
df_alfa_agosto_2024_queretaro.columns

Index(['id', 'Categoria', 'Precio', 'propiedad', 'metros_total',
       'metros_construido', 'precio_m2_terreno', 'precio_m2_construido',
       'tiempo_de_publicacion', 'Estado', 'Tipo', 'Estacionamientos',
       'Recamaras', 'Banos', 'Medio_banos', 'Banos_Total', 'Seguridad',
       'Colonia', 'CP', 'url'],
      dtype='object')

In [9]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    # Verificar si la columna 'm2_contruido' existe en el DataFrame
#   Renombrar la columna
    df.rename(columns={'metros_construido':'m2_construido','m2_contruido': 'm2_construido','Categoria':'categoria','segmento':'categoria'}, inplace=True)
    # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df        

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [10]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
dataframes_name = []

for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id', 'categoria','m2_construido']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)
     # Componer el nombre del DataFrame en la lista dataframes_name
    dataframes_name.append(nombre_df)


# Cantidad de registros por metros cuadrados construidos, por segmento

In [11]:
def procesar_dataframes(dataframes_list, dataframes_name):
    for i, df in enumerate(dataframes_list):
        # Crear una nueva columna con la primera letra de la categoría
        df['segmento'] = df['categoria'].str[0]
        df.drop(columns=['categoria'], inplace=True)

        # Clasificar 'baño_total' en rangos específicos
        bins = [0, 100, 150, 200, 250, 300, float('inf')]
        labels = ['0-100', '100-150', '150-200', '200-250', '250-300', '300+']
        df['m2_construido'] = pd.cut(df['m2_construido'], bins=bins, labels=labels, right=False)

        # Agrupar y contar los datos
        tabla = df.groupby(['segmento', 'm2_construido']).size().unstack(fill_value=0)

        # Imprimir el nombre del DataFrame y la tabla resultante
        print(f"Nombre del DataFrame: {dataframes_name[i]}")
        print(tabla)

# Procesar los DataFrames de la lista
procesar_dataframes(dataframes_list, dataframes_name)

Nombre del DataFrame: df_alfa_abril_2024_queretaro
m2_construido  0-100  100-150  150-200  200-250  250-300  300+
segmento                                                      
A                 85      307      352      213       47    10
B                215      530      310       57       13     4
C                145      330       69       12        9    10
D                296       70       28       20       13    24
E                 77       62       50       46       45   296
L                  0        0        1        1       13   429
S                 26      395      607      986      870  1692
Nombre del DataFrame: df_alfa_agosto_2024_queretaro
m2_construido  0-100  100-150  150-200  200-250  250-300  300+
segmento                                                      
A                 98      377      458      258       52    21
B                215      597      333       68        4     9
C                226      382       74       20       16    14
D              

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11108\275059014.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['segmento'] = df['categoria'].str[0]
C:\Users\yoe11\AppData\Local\Temp\ipykernel_11108\275059014.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['categoria'], inplace=True)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_11108\275059014.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

## Tabla general de frecuencia
 Recuento total de filas para cada combinación de valores únicos de 'segmento' y 'm2_contruido' con todos los DataFrame

In [12]:
# Crear una nueva función para sumar las tablas de todos los DataFrames
def sumar_tablas(dataframes_list):
    # Inicializar una tabla vacía para almacenar la suma
    tabla_suma = None
    
    # Iterar sobre cada DataFrame en la lista
    for df in dataframes_list:
        # Agrupar y contar los datos en el DataFrame actual
        tabla_actual = df.groupby(['segmento', 'm2_construido']).size().unstack(fill_value=0)
        
        # Sumar la tabla actual al acumulador (tabla_suma)
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    # Imprimir la tabla resultante de la suma
    print("Tabla general de frecuencia:")
    print(tabla_suma)

# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
sumar_tablas(dataframes_list)

Tabla general de frecuencia:
m2_construido   0-100  100-150  150-200  200-250  250-300    300+
segmento                                                         
A               407.0   1574.0   2356.0   1402.0    281.0    77.0
B               987.0   3099.0   2159.0    408.0     56.0    24.0
C              1071.0   1911.0    418.0     78.0     37.0    42.0
D              1671.0    338.0    107.0     76.0     40.0    79.0
E               551.0    356.0    266.0    207.0    170.0  1310.0
L                 5.0      0.0      7.0      6.0     45.0  1697.0
S               163.0   1686.0   2660.0   4503.0   4152.0  7550.0


C:\Users\yoe11\AppData\Local\Temp\ipykernel_11108\470711682.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tabla_actual = df.groupby(['segmento', 'm2_construido']).size().unstack(fill_value=0)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_11108\470711682.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tabla_actual = df.groupby(['segmento', 'm2_construido']).size().unstack(fill_value=0)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_11108\470711682.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain curren

In [13]:
def sumar_tablas(dataframes_list):
    tabla_suma = None
    
    for df in dataframes_list:
        tabla_actual = df.groupby(['m2_construido', 'segmento']).size().unstack(fill_value=0)
        
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    print("Tabla general de frecuencia:")
    print(tabla_suma)
    
    fig = graficar_tabla(tabla_suma)
    
    # Guardar la gráfica como archivo HTML
    guardar_grafico_como_html(fig, 'g_bar_dist_m2_construidos', carpeta='assets/graficas')
    
    return fig

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

def graficar_tabla(tabla):  
    segmentos = tabla.index
    banos = tabla.columns
    
    fig = go.Figure()
    
    for i, segmento in enumerate(segmentos):
        color = colores[i % len(colores)]
        fig.add_trace(go.Bar(
            x=banos,
            y=tabla.loc[segmento],
            name=segmento,
            marker_color=color
        ))
    
    fig.update_layout(
        xaxis_title="Segmento",        
        yaxis=dict(
            title='Número de casas',
            gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
            gridwidth=1,
        ),
        legend_title="Rango de M2",
        barmode='group',
        legend=dict(
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
        ),
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
        margin=dict(l=10, r=10, t=10, b=10) # Ajusta los márgenes (left, right, top, bottom)
    )    
    
    fig.show()    
    return fig

def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    ruta_archivo = os.path.join(carpeta, f'{nombre_archivo}.html')
    pio.write_html(fig, ruta_archivo)
# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
fig = sumar_tablas(dataframes_list)

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11108\3114147233.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tabla_actual = df.groupby(['m2_construido', 'segmento']).size().unstack(fill_value=0)


Tabla general de frecuencia:
segmento          A     B     C     D     E       L     S
m2_construido                                            
0-100           407   987  1071  1671   551     5.0   163
100-150        1574  3099  1911   338   356     0.0  1686
150-200        2356  2159   418   107   266     7.0  2660
200-250        1402   408    78    76   207     6.0  4503
250-300         281    56    37    40   170    45.0  4152
300+             77    24    42    79  1310  1697.0  7550


C:\Users\yoe11\AppData\Local\Temp\ipykernel_11108\3114147233.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tabla_actual = df.groupby(['m2_construido', 'segmento']).size().unstack(fill_value=0)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_11108\3114147233.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tabla_actual = df.groupby(['m2_construido', 'segmento']).size().unstack(fill_value=0)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_11108\3114147233.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain cur

In [14]:
import pandas as pd
import plotly.graph_objects as go
import os
import plotly.io as pio

# Definir el orden deseado de los segmentos
orden_segmentos = ["E", "D", "C", "B", "A", "S", "L", 'ELITE']

def sumar_tablas(dataframes_list):
    tabla_suma = None
    
    for df in dataframes_list:
        # Agrupar y contar casas según 'm2_construido' y 'segmento'
        tabla_actual = df.groupby(['segmento', 'm2_construido']).size().unstack(fill_value=0)
        
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    print("Tabla general de frecuencia:")
    print(tabla_suma)
    
    # Reordenar los índices (segmentos) de acuerdo a 'orden_segmentos'
    tabla_suma = tabla_suma.reindex(orden_segmentos).fillna(0)
    
    # Graficar la tabla reordenada
    fig = graficar_tabla(tabla_suma)
    
    # Guardar la gráfica como archivo HTML
    guardar_grafico_como_html(fig, 'g_bar_dist_m2_construidos', carpeta='assets/graficas')
    
    return fig

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

def graficar_tabla(tabla):  
    # Obtenemos las columnas (m2_construido) y los índices (segmentos) de la tabla
    segmentos = tabla.index
    m2_construido = tabla.columns
    
    fig = go.Figure()
    
    # Trazar las barras para cada rango de m2_construido dentro de cada segmento
    for i, m2 in enumerate(m2_construido):
        color = colores[i % len(colores)]
        fig.add_trace(go.Bar(
            x=segmentos,            # Segmentos en el eje X
            y=tabla[m2],            # Cantidad de casas para ese rango de m2_construido
            name=f"{m2}",        # Nombre del rango de m2_construido
            marker_color=color
        ))
    
    fig.update_layout(
        xaxis_title="Segmento",        
        yaxis=dict(
            title='Número de casas',
            gridcolor='#dddcda',  # Color de las líneas que dividen los rangos del eje Y
            gridwidth=1,
        ),
        legend_title="Rango M2 construido",
        barmode='group',  # Las barras se agrupan por rango de m2_construido
        legend=dict(
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
        ),
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
        margin=dict(l=10, r=10, t=10, b=10)  # Ajusta los márgenes (left, right, top, bottom)
    )    
    
    fig.show()    
    return fig

def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    ruta_archivo = os.path.join(carpeta, f'{nombre_archivo}.html')
    pio.write_html(fig, ruta_archivo)

# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
fig = sumar_tablas(dataframes_list)


Tabla general de frecuencia:
m2_construido   0-100  100-150  150-200  200-250  250-300    300+
segmento                                                         
A               407.0   1574.0   2356.0   1402.0    281.0    77.0
B               987.0   3099.0   2159.0    408.0     56.0    24.0
C              1071.0   1911.0    418.0     78.0     37.0    42.0
D              1671.0    338.0    107.0     76.0     40.0    79.0
E               551.0    356.0    266.0    207.0    170.0  1310.0
L                 5.0      0.0      7.0      6.0     45.0  1697.0
S               163.0   1686.0   2660.0   4503.0   4152.0  7550.0


C:\Users\yoe11\AppData\Local\Temp\ipykernel_11108\2577868118.py:14: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11108\2577868118.py:14: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11108\2577868118.py:14: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11108\2577868118.py:14: FutureWarning:

In [16]:
def sumar_tablas(dataframes_list):
    tabla_suma = None
    
    for df in dataframes_list:
        tabla_actual = df.groupby(['m2_construido', 'segmento']).size().unstack(fill_value=0)
        
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    print("Tabla general de frecuencia:")
    print(tabla_suma)
    
    fig = graficar_tabla(tabla_suma)
    
    # Guardar la gráfica como archivo HTML
    guardar_grafico_como_html(fig, 'g_pie_dist_m2_construidos', carpeta='assets/graficas')
    
    return fig

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

def graficar_tabla(tabla):
    # Sumar los valores por 'm2_contruido' para obtener el total por rango de M2
    suma_por_m2 = tabla.sum(axis=1)
    
    fig = go.Figure(data=[go.Pie(
        labels=suma_por_m2.index.astype(str),
        values=suma_por_m2.values,
        marker=dict(colors=colores),
        textinfo='percent',
        insidetextorientation='radial'
    )])
    
    fig.update_layout(
        # title="Distribución de M2 construidos",
        legend_title="Rango M2 construido",
        barmode='group',
        legend=dict(
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02, 
            xanchor='right',
            x=0.72
        ),
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
        margin=dict(l=10, r=10, t=10, b=10) # Ajusta los márgenes (left, right, top, bottom)
    )    
    
    fig.show()
    return fig

def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    ruta_archivo = os.path.join(carpeta, f'{nombre_archivo}.html')
    pio.write_html(fig, ruta_archivo)

fig1 = sumar_tablas(dataframes_list)

Tabla general de frecuencia:
segmento          A     B     C     D     E       L     S
m2_construido                                            
0-100           407   987  1071  1671   551     5.0   163
100-150        1574  3099  1911   338   356     0.0  1686
150-200        2356  2159   418   107   266     7.0  2660
200-250        1402   408    78    76   207     6.0  4503
250-300         281    56    37    40   170    45.0  4152
300+             77    24    42    79  1310  1697.0  7550


C:\Users\yoe11\AppData\Local\Temp\ipykernel_11108\1836979870.py:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11108\1836979870.py:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11108\1836979870.py:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11108\1836979870.py:5: FutureWarning:

Th